In [ ]:
#default_exp filter
%load_ext autoreload
%autoreload 2

# Filtragem
> Este módulo concentra funções auxiliares específicas que filtram os dados do banco com campos e formatação de interesse para aplicações específicas como o [appAnalise](https://github.com/EricMagalhaesDelgado/appAnalise) por exemplo.

In [ ]:
#export
from pathlib import Path
import pandas as pd
from datetime import datetime
from anateldb.query import *
from anateldb.constants import console
from unidecode import unidecode
from fastcore.test import *
from fastcore.script import call_parse, Param, store_true, bool_arg

In [ ]:
#export
@call_parse
def formatar_db(
    path: Param("Pasta onde salvar os arquivos", str),
    atualizar: Param("Atualizar as bases de dados?", bool_arg) = True,
) -> None:
    dest = Path(path)
    dest.mkdir(parents=True, exist_ok=True)
    time = datetime.today().strftime("%d/%m/%Y %H:%M:%S")
    stel = read_stel(path, atualizar).sort_values("Frequência").loc[:, TELECOM]
    stel.rename(
        columns={"Serviço": "Num_Serviço", "Número da Estação": "Número_da_Estação"},
        inplace=True,
    )
    radcom = read_radcom(path, atualizar)
    radcom.rename(columns={"Número da Estação": "Número_da_Estação"}, inplace=True)
    mosaico = read_mosaico(path, atualizar)
    with console.status(
        "[blue]Formatação e filtro das diversas colunas ...", spinner="pong"
    ) as status:
        radcom["Num_Serviço"] = "231"
        radcom["Status"] = "RADCOM"
        radcom["Classe"] = radcom.Fase.str.strip() + "-" + radcom.Situação.str.strip()
        radcom["Entidade"] = radcom.Entidade.str.rstrip().str.lstrip()
        radcom["Num_Ato"] = ""
        radcom["Data_Ato"] = ""
        radcom["Validade_RF"] = ""
        radcom = radcom.loc[:, RADIODIFUSAO]
        mosaico = mosaico.loc[:, RADIODIFUSAO]
        rd = mosaico.append(radcom)
        stel["Num_Ato"] = ""
        stel["Data_Ato"] = ""
        stel["Validade_RF"] = ""  # FIXME
        stel['Entidade'] = stel.Entidade.str.rstrip().str.lstrip()

        rd["Descrição"] = (
            rd.Status.astype(str)
            + ", "
            + rd.Classe.astype(str)
            + ", "
            + rd.Entidade.astype(str).str.title()
            + " ("
            + rd.Fistel.astype(str)
            + ", "
            + rd["Número_da_Estação"].astype(str)
            + "), "
            + rd.Município.astype(str)
            + "/"
            + rd.UF.astype(str)
        )
        stel["Descrição"] = (
            stel.Num_Serviço.astype(str)
            + ", "
            + stel.Entidade.astype(str).str.title()
            + " ("
            + stel.Fistel.astype(str)
            + ", "
            + stel["Número_da_Estação"].astype(str)
            + "), "
            + stel["Município"].astype(str)
            + "/"
            + stel.UF.astype(str)
        )
        export_columns = [
            "Frequência",
            "Latitude",
            "Longitude",
            "Descrição",
            "Num_Serviço",
            "Número_da_Estação",
            "Num_Ato",
            "Data_Ato",
            "Validade_RF",
        ]
        rd = rd.loc[:, export_columns]
        stel = stel.loc[:, export_columns]
        rd = rd.append(stel).sort_values("Frequência").reset_index(drop=True)
        rd.columns = APP_ANALISE
#        rd = rd.fillna('')

    with console.status("[magenta]Salvando os arquivos ...", spinner="grenade") as status:

        date = pd.DataFrame(columns=[time])
        #date["Day"] = [str(datetime.today()]
        #date["Time"] = [str(datetime.today().time())]

        with pd.ExcelWriter(f"{dest}/AnatelDB.xlsx") as workbook:
            date.to_excel(workbook, sheet_name="ExtractDate", index=False)
            rd.to_excel(workbook, sheet_name="DataBase", index=False)


    console.print("Sucesso :zap:")

In [ ]:
formatar_db(r'D:\OneDrive - ANATEL\BaseDados', atualizar=False)

Output()

Output()

Sucesso ⚡

In [ ]:
#pb = read_plano_basico(r'D:\OneDrive - ANATEL\BaseDados\Canais.zip')

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted constants.ipynb.
Converted filter.ipynb.
Converted index.ipynb.
Converted queries.ipynb.
